Import of dependencies

In [ ]:
import pandas as pd
import nltk
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt 
import seaborn as sns
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, MiniBatchNMF, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from google.colab import drive
drive.mount('/content/drive')

Helper functions

In [53]:
def remove_by_regex(texts, regexp):
        output_texts = regexp.sub("",texts)
        return output_texts

def remove_urls(input_text):
        return remove_by_regex(input_text, re.compile(r"http.?://[^\s]+[\s]?"))

def remove_hashtags_and_mentions(input_text):
        input_text = remove_by_regex(input_text, re.compile(r"(#[A-Za-z]+[A-Za-z0-9-_]+)"))
        return remove_by_regex(input_text, re.compile(r"(?<!RT\s)(@[A-Za-z]+[A-Za-z0-9-_]+)"))

def preprocesstextForLDA(incorpus):
    outcorpus = []
    for i in range(0, len(incorpus)):
      corpuselement = remove_urls(incorpus[i])
      corpuselement = remove_hashtags_and_mentions(incorpus[i])
      corpuselement = re.sub('[^a-zA-Z]', ' ', corpuselement)
      corpuselement = corpuselement.lower()
      corpuselement = corpuselement.split()
      ps = PorterStemmer()
      all_stopwords = stopwords.words('english')
      all_stopwords.append('https')
      all_stopwords.append('amp')
      corpuselement = [word for word in corpuselement if not word in set(all_stopwords)]
      corpuselement = ' '.join(corpuselement)
      outcorpus.append(corpuselement)
    return outcorpus

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

def preprocesstextForLDANews(incorpus):
    outcorpus = []
    for i in range(0, len(incorpus)):
      corpuselement = remove_urls(incorpus[i])
      corpuselement = remove_hashtags_and_mentions(incorpus[i])
      corpuselement = re.sub('[^a-zA-Z]', ' ', corpuselement)
      corpuselement = corpuselement.lower()
      corpuselement = corpuselement.split()
      ps = PorterStemmer()
      all_stopwords = stopwords.words('english')
      all_stopwords.append('ii')
      all_stopwords.append('matin')
      all_stopwords.append('elizabeth')
      all_stopwords.append('elizabeths')
      all_stopwords.append('queens')
      corpuselement = [word for word in corpuselement if not word in set(all_stopwords)]
      corpuselement = ' '.join(corpuselement)
      outcorpus.append(corpuselement)
    return outcorpus

def uniqueWords(x):
  words = ' '.join([text for text in x]) 
  words = nltk.tokenize.word_tokenize(words)
  fdist1 = nltk.FreqDist(words)
  freq = dict((word, freq) for word, freq in fdist1.items() if not word.isdigit())
  return freq

In [36]:
dict=np.load("/content/drive/MyDrive/A3/ABCbackupFinal.npy", allow_pickle='TRUE').item()

In [37]:
len(dict.keys()) #84

84

In [ ]:
contentlist=[]
totalwords = 0
minwords=999
maxwords = 0
unique=[]
for key,value in dict.items():
  for text in dict[key]:
    if not text.lower() in contentlist:
      contentlist.append(text.lower())
      thiswords = 0
      for word in text.lower().split():
        thiswords+=1
        if not word in unique:
          unique.append(word)
      if thiswords < minwords:
        minwords=thiswords
      if thiswords > maxwords:
        maxwords = thiswords
      totalwords += thiswords

print('Min: ', minwords)
print('Max: ', maxwords)
print('Max: ', totalwords/len(contentlist))
print('Unq: ', len(unique))

In [ ]:
contentlist

In [ ]:
unique

In [39]:
len(contentlist)

91

In [ ]:
contentlist

In [66]:
bowProc = preprocesstextForLDANews(contentlist)

In [ ]:
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=500, stop_words="english"
)
tf = tf_vectorizer.fit_transform(bowProc)

lda = LatentDirichletAllocation(
    n_components=10,
    max_iter=8,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
)
lda.fit(tf)
tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, 8, "")